In [4]:
import numpy as numpy
from matplotlib import pyplot as matplot
%matplotlib inline
import pandas as pandas
import math
from sklearn import preprocessing
from sklearn import svm
import cv2
import os
import ntpath
from sklearn.metrics.pairwise import cosine_similarity
import fnmatch
import math
from math import sqrt,log
from cmath import sqrt

data = []
Filename = []
Person = []
angles = []
Len = []
Rho = []


def crop_image(img,tol=0):
    mask = img>tol
    return img[numpy.ix_(mask.any(1),mask.any(0))]
def thinning(img):
    #img = cropimage
    size = numpy.size(img)
    skel = numpy.zeros(img.shape,numpy.uint8)

    ret,img = cv2.threshold(img,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False

    while( not done):
        eroded = cv2.erode(img,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img,temp)
        skel = cv2.bitwise_or(skel,temp)
        img = eroded.copy()

        zeros = size - cv2.countNonZero(img)
        if zeros==size:
            done = True

  #  plt.imshow(skel,'gray')
   # plt.show()
    return skel
def coords(thinnedimage):
    rows,cols=thinnedimage.shape
    img_topleft=thinnedimage[0:int(rows/2),0:int(cols/2)]
    img_topright=thinnedimage[int(rows/2)+1:rows,0:int(cols/2)]
    img_bottomleft=thinnedimage[0:int(rows/2),int(cols/2)+1:cols]
    img_bottomright=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:cols]
    
    topleft_x,topleft_y=COG(img_topleft)
    topright_x,topright_y=COG(img_topright)
    bottomleft_x,bottomleft_y=COG(img_bottomleft)
    bottomright_x,bottomright_y=COG(img_bottomright)

    return topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y

def COG(img):
    x_cor=0
    xrun_sum=0
    y_cor=0
    yrun_sum=0
    #print(img.shape)
    for i in range(img.shape[0]):
        x_cor+=sum(img[i])*i/255
        xrun_sum+=sum(img[i])/255

    for i in range(img.shape[1]):
        y_cor+=sum(img[:,i])*i/255
        yrun_sum+=sum(img[:,i])/255
        #print(img.shape[1]) 
        if yrun_sum==0:
            x_pos=0
        else:
            x_pos=y_cor/(yrun_sum)
        if xrun_sum==0:
            y_pos=0
        else:
            y_pos=x_cor/(xrun_sum)
        
   # print(x_pos)
  #  print(y_pos)
    
    return (x_pos/img.shape[1],y_pos/img.shape[0])

def tan_i(x):
    #print(x)
    if x[0]==0:
        return 90
    return math.degrees(math.atan(x[1]/x[0]))

def tan(thinnedimage):
    rows,cols=thinnedimage.shape
    
    img_tl1=thinnedimage[0:int(rows/2),0:int(cols/4)]
    img_tl2=thinnedimage[0:int(rows/2),int(cols/4)+1:int(cols/2)]
    
    img_tr1=thinnedimage[0:int(rows/2),int(cols/2)+1:int(0.75*cols)]
    img_tr2=thinnedimage[0:int(rows/2),int(0.75*cols)+1:cols]
    
    img_bl1=thinnedimage[int(rows/2)+1:rows,0:int(cols/4)]
    img_bl2=thinnedimage[int(rows/2)+1:rows,int(cols/4)+1:int(cols/2)]
    
    img_br1=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:int(0.75*cols)]
    img_br2=thinnedimage[int(rows/2)+1:rows,int(0.75*cols)+1:cols]
    

    #plt.imshow(timg,'gray')
    #plt.show()
    
    tl1=tan_i(COG(img_tl1))
    tl2=tan_i(COG(img_tl2))
    tr1=tan_i(COG(img_tr1))
    tr2=tan_i(COG(img_tr2))
    bl1=tan_i(COG(img_bl1))
    bl2=tan_i(COG(img_bl2))
    br1=tan_i(COG(img_br1))
    br2=tan_i(COG(img_br2))
    
    #plt.imshow(img_br1,'gray')
    #plt.show()
    #print(COG(img_br1))
    return tl1,tl2,tr1,tr2,bl1,bl2,br1,br2

def leng_rho(img):
    kernel = numpy.ones((2,2), numpy.uint8)
    img_dilation = cv2.dilate(img, kernel, iterations=2)
    img_erosion = cv2.erode(img, kernel, iterations=4)
    edges = cv2.Canny(img_erosion,cv2.THRESH_BINARY+cv2.THRESH_OTSU,550,apertureSize = 3)
    minLineLength=100
    maxLineGap=10
    threshold=20
    image=edges
    lines = cv2.HoughLinesP(image=img,rho=1,theta=numpy.pi/180, threshold=threshold,lines=numpy.array([]), minLineLength=minLineLength,maxLineGap=maxLineGap)
    for x1,y1,x2,y2 in lines[0]:
        theta = math.atan(float(y2 - y1) / (x2 - x1))
        Leng = sqrt((x2-x1)^2 + (y2-y1)^2)
        Rho = (x1 * y2 - x2 * y1) / Leng
        angles.append({'theta': theta, 'p1': (x1, y1), 'p2': (x2, y2)})
    #for i in range(len(angles)):
     #   for j in range(i + 1, len(angles)):
      #      delta = abs(angles[i]['theta'] - angles[j]['theta'])               
    a,b,c = lines.shape
    theta = round(theta,4)
    Rho = round(abs(Rho),0)
    Leng = round(abs(Leng),0)
    return a,b,c,theta,Rho,Leng

def createfingerprint_Genuine(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    a,b,c,theta,Rho,Leng = leng_rho(image)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    
    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
      
    a=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio, a,b,c,theta,Rho,Leng],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio","a","b","c","theta","Rho","Leng"])
    data.append(a)

def create_training_Genuine_dataset(m):
    numpy.seterr(divide='ignore', invalid='ignore')
    os.chdir(Genuinedir)
    filecount = len(fnmatch.filter(os.listdir(), '*.png'))
    n = int(filecount)
    for j in range(1,m+1):
        for i in range(1,n+1):
            path = f"C:\\Siddhartha\\Original\\00{j}_{i}.png"
            fn = ntpath.basename(path)
            Filename.append(fn)
            tn = j
            Person.append(tn)
            image = cv2.imread(path,0)
            createfingerprint_Genuine(image)
            Original_Signature_Table=pandas.DataFrame(data)            
    #return Original_Signature_Table
    Original_Signature_Table['Filename'] = Filename
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Person'] = Person
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Category'] = "Genuine"
    Original_Signature_Table.to_csv('Training.csv', index = False)
    Summary_Table = pandas.DataFrame(Original_Signature_Table.describe()).T
    Summary_Table.to_csv('Training_Summary.csv', index = False)
    Original_Signature_Table
    print('Preprocessing of Training Data Completed')
    return Original_Signature_Table

In [5]:
Genuinedir = 'C:\\Siddhartha\\Original'
create_training_Genuine_dataset(1)

Preprocessing of Training Data Completed


,Person,Filename,Norm_area,connected_comps,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,...,tan_bottomright_x,tan_bottomright_y,Aspect_Ratio,a,b,c,theta,Rho,Leng,Category
0,1,001_1.png,0.139030,8.0,0.564356,0.768614,0.522456,0.385612,0.582405,0.622935,...,37.537878,26.970577,1.367133,336.0,1.0,4.0,-0.5758,5593.000000,20.0,Genuine
1,1,001_2.png,0.152696,10.0,0.434761,0.853383,0.430759,0.228998,0.514960,0.730785,...,23.209179,46.538274,3.478873,180.0,1.0,4.0,-0.0183,1359.000000,22.0,Genuine
2,1,001_3.png,0.193033,5.0,0.534267,0.477112,0.706160,0.436912,0.558845,0.523014,...,30.179978,61.510418,1.090909,223.0,1.0,4.0,-0.6802,6993.000000,7.0,Genuine
3,1,001_4.png,0.109489,6.0,0.433331,0.660764,0.751744,0.736111,0.498063,0.535797,...,52.134249,41.201171,4.268116,168.0,1.0,4.0,0.1569,77.000000,23.0,Genuine
4,1,001_5.png,0.078026,3.0,0.502653,0.549889,0.190750,0.422015,0.522454,0.283367,...,90.000000,90.000000,2.272727,92.0,1.0,4.0,-0.4005,2078.000000,13.0,Genuine
5,1,001_6.png,0.137314,7.0,0.365701,0.655345,0.508185,0.376421,0.419628,0.505713,...,31.932419,37.766817,1.784314,231.0,1.0,4.0,0.3658,1142.000000,22.0,Genuine
6,1,001_7.png,0.178588,7.0,0.626979,0.630058,0.581296,0.377583,0.435630,0.568155,...,32.534148,16.032168,1.733333,141.0,1.0,4.0,0.5565,626.000000,9.0,Genuine
7,1,001_8.png,0.071511,4.0,0.564963,0.608238,0.256198,0.390940,0.404541,0.579666,...,90.000000,90.000000,1.777070,176.0,1.0,4.0,0.7854,inf,0.0,Genuine
8,1,001_9.png,0.097953,7.0,0.608711,0.532974,0.637812,0.326156,0.423755,0.624146,...,41.252877,25.729372,1.948718,241.0,1.0,4.0,0.1740,3090.000000,20.0,Genuine
9,1,001_10.png,0.120362,4.0,0.491162,0.730149,0.465268,0.305605,0.574987,0.639874,...,19.888122,14.093792,3.267717,146.0,1.0,4.0,0.0867,1378.000000,21.0,Genuine
